In [1]:
import os
import tensorflow as tf
from tensorflow import keras as keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models, losses, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, Input, BatchNormalization, Dropout
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


In [3]:
from google.colab import drive
drive.mount('cnontent/')

Mounted at cnontent/


In [5]:
cd /content/cnontent/MyDrive/SSU/nural_nework/project

/content/cnontent/MyDrive/SSU/nural_nework/project


In [7]:
def convert_data_csv_to_numpy(data, sorting = False): # pandas 통해 읽은 csv data numpy 형태로 변경
  if sorting == True:
    data = data.sort_values(by=[0], axis=0)

  label = np.array(data[0]) # csv file 에서 0번째 colum은 index임
  only_data = np.array(data.drop([0], axis = 1)).reshape((-1, 28, 28, 1)) # csv file에서 0번 째 colum 탈락 -> data만 남게 됨
  # only_data = tf.convert_to_tensor(only_data, dtype=tf.float32)
  return only_data, label

In [8]:
train_df = pd.read_csv("./emnist-byclass-train.csv", header=None)
test_df = pd.read_csv("./emnist-byclass-test.csv", header=None)

In [9]:
X_train, y_train  = convert_data_csv_to_numpy(train_df)
X_test, y_test  = convert_data_csv_to_numpy(test_df)

train_df = None
test_df = None

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X_train/255., y_train, test_size=0.2, random_state=42)

In [11]:
raw_y_train = y_train.copy()
raw_y_val = y_val.copy()
raw_y_test = y_test.copy()

y_train = tf.keras.utils.to_categorical(y_train)
y_val = tf.keras.utils.to_categorical(y_val)
y_test = tf.keras.utils.to_categorical(y_test)

In [12]:
# numpy type의 data tf.tensor type으로 변경
train_data = tf.convert_to_tensor(X_train, dtype=tf.float32)
val_data = tf.convert_to_tensor(X_val, dtype=tf.float32)
test_data = tf.convert_to_tensor(X_test, dtype=tf.float32)

In [13]:
print(train_data.shape, val_data.shape, test_data.shape)

(558345, 28, 28, 1) (139587, 28, 28, 1) (116323, 28, 28, 1)


In [14]:
# 1채널 gray 영상 3채널로 변경 -> pretrained model이 3채널 영상 사용하기 때문
final_train = tf.image.grayscale_to_rgb(train_data)
train_data = []

final_val = tf.image.grayscale_to_rgb(val_data)
val_data = []

final_test = tf.image.grayscale_to_rgb(test_data)
test_data = []

In [15]:
print(final_train.shape, final_val.shape, final_test.shape)

(558345, 28, 28, 3) (139587, 28, 28, 3) (116323, 28, 28, 3)


In [16]:
# mobilenet predtrained model 다운로드
mobile2 = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape = (224, 224, 3), include_top=False)

9420800/9406464 [==============================] - 0s 0us/step


In [17]:
# mobilenoet v2 파라미터 변경 baseline(test_acc : 0.8805)
# Mobile_ReLU_SGD_800_05.h5

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(62, activation = 'softmax')

optimizer = keras.optimizers.SGD(learning_rate = 0.05)

mobilev2_best_model = keras.models.Sequential()
# mobilev2_best_model.add(tf.keras.layers.Normalization( axis=-1, mean=44.412914, variance=84.77896, input_shape = (28, 28, 3)))
mobilev2_best_model.add(tf.keras.layers.Resizing(height = 224, width = 224, input_shape = (28, 28, 3)))
mobilev2_best_model.add(mobile2)
mobilev2_best_model.add(global_average_layer)
mobilev2_best_model.add(tf.keras.layers.Dense(800))
mobilev2_best_model.add(tf.keras.layers.LeakyReLU())
mobilev2_best_model.add(tf.keras.layers.Dense(100))
mobilev2_best_model.add(tf.keras.layers.LeakyReLU())
mobilev2_best_model.add(prediction_layer)

mobilev2_best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 224, 224, 3)       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_1 (Dense)             (None, 800)               1024800   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 800)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               8

In [18]:
# 모델 compile 및 training
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
mobilev2_best_model.compile(loss="categorical_crossentropy",
                optimizer=optimizer,
                metrics=["accuracy"])
best_history = mobilev2_best_model.fit(final_train, y_train, batch_size = 64, epochs=10, validation_data=(final_val, y_val), callbacks=[early_stopping_cb])

Epoch 1/10
  99/8725 [..............................] - ETA: 42:14 - loss: 1.5532 - accuracy: 0.6061

KeyboardInterrupt: ignored

In [ ]:
# 모델 eval
mobilev2_best_model.evaluate(final_test/255., y_test)

In [ ]:
# 가능하면 모델 파일 저장도 부탁드립니다!!
mobilev2_best_model.save("submit_emnist_mobilev2.h5", save_format="h5")